In [1]:
!pip install pythainlp

In [1]:
import deepcut
import pandas as pd
import numpy as np
import re
from itertools import chain
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from flask import Flask, jsonify, request
import threading 
import time
import json
import requests

## data from all category

In [2]:
from itertools import chain
def tokenize_text_list(ls):
    """Tokenize list of text"""
    return list(chain.from_iterable([deepcut.tokenize(ls)]))

In [3]:
exit_flag = False
beforeTok={}

def getQuestions():
    global beforeTok
    while True:
#         raw_questions = backendAPI()
        raw_questions = getQuestionsFromBackendAPI() # <-- Uncomment this to get real questions
        
        # do tokenize
        tokenize_questions = raw_questions
        
        curriculumDF = pd.DataFrame(data=tokenize_questions['หลักสูตร'])
        curriculumDF = curriculumDF.rename(columns={0:"curriculum"})
        
        admissionDF = pd.DataFrame(data=tokenize_questions['การรับเข้านักศึกษา'])
        admissionDF = admissionDF.rename(columns={0:"admission"})
        
        enrollmentDF = pd.DataFrame(data=tokenize_questions['ลงทะเบียนเรียน'])
        enrollmentDF = enrollmentDF.rename(columns={0:"enrollment"})
        
        faqDF = pd.DataFrame(data=tokenize_questions['คำถามทั่วไป'])
        faqDF = faqDF.rename(columns={0:"faq"})
        
        tokenized_enrollment =enrollmentDF.enrollment.map(tokenize_text_list)
        tokenized_admission =admissionDF.admission.map(tokenize_text_list)
        tokenized_curriculum =curriculumDF.curriculum.map(tokenize_text_list)
        tokenized_faq =faqDF.faq.map(tokenize_text_list)
        
#         beforeTok = {}
        beforeTok['ลงทะเบียนเรียน'] = enrollmentDF
        beforeTok['การรับเข้านักศึกษา'] = admissionDF
        beforeTok['หลักสูตร'] = curriculumDF
        beforeTok['คำถามทั่วไป'] = faqDF
        
        # Update question
        questions_data = tokenize_questions
        if exit_flag: 
            break
        
        # Set query time
        time.sleep(259200)

In [4]:
url = 'https://natthawat.live/api'

In [5]:
def getQuestionsFromBackendAPI():
    response = requests.get('%s/km/faq' % url)
    faqs = json.loads(response.text)

    response = requests.get('%s/km/category' % url)
    categories = json.loads(response.text)

    questions_data = {}
    for category in categories:
        questions_data[category['category']] = []
        for faq in faqs:
            if faq['category']['category'] == category['category']:
                questions_data[category['category']].append(faq['question'])
                
    return questions_data

In [6]:
getQuestionsThread = threading.Thread(target = getQuestions)

In [7]:
getQuestionsThread.start()

In [8]:
while len(beforeTok) is 0:
    continue
print(beforeTok)

{'ลงทะเบียนเรียน':                                            enrollment
0   สามารถลงทะเบียนเกินหน่วยกิตที่หลักสูตรกำหนดได้...
1    ใบแจ้งชำระเงินค่าลงทะเบียน สั่งพิมพ์ได้จากที่ไหน
2                        ตอนดรอปต้องขอลายเซ็นใครบ้าง?
3                       ขอทรานสคริปต์ออนไลน์ได้ที่ไหน
4                     ยังต้องเข้าไปเรียนที่มหาลัยไหม?
5       ลงทะเบียนเรียนเพิ่มรายวิชา ชำระเงินได้ที่ไหน?
6   ชำระเงินค่าลงทะเบียนแล้ว แต่ในระบบยังขึ้นว่าไม...
7   ถ้าจะดึงบางวิชาลงมาเรียนแทนตัวที่ติด f ได้ไหม ...
8   ลืมจ่ายเงินค่าลงทะเบียน และ ใบแจ้งหนี้เกินกำหน...
9   ถ้าต้องการลงทะเบียนเรียนน้อยกว่า 9 หน่วยกิตต้อ...
10                   ถ้าการย้ายหลักสูตรต้องทำอย่างไร?
11                            ตรวจวันเวลาสอบได้ที่ไหน
12  ขอลดรายวิชาออนไลน์ หลังหมดเขต เพิ่ม-ลด 2 สัปดา...
13  กรณีเพิ่มและหลักสูตรปิดรายวิชา จะได้รับเงินคืน...
14                      แจ้งขอสำเร็จการศึกษาได้ที่ไหน
15  ถ้าต้องการลงทะเบียนเรียนมากกว่า 19 หน่วยกิตต้อ...
16                               วิชาเลือกไหนน่าสนใจ?
17       

In [13]:
keys_list = list(beforeTok)

In [15]:
len(keys_list)

4

## category

In [22]:
data = {'Category': keys_list}
data = pd.DataFrame(data=data)
data
# data = pd.read_excel("Category.xlsx")

,Category
0,ลงทะเบียนเรียน
1,การรับเข้านักศึกษา
2,หลักสูตร
3,คำถามทั่วไป


In [23]:
#Lib
import pickle
#Load File
with open('token_text_category.data', 'rb') as filehandle:
    # read the data as binary data stream
    tokenized_texts = pickle.load(filehandle)

In [24]:
def text_to_bow(tokenized_text, vocabulary_):
    n_doc = len(tokenized_text)
    values, row_indices, col_indices = [], [], []
    for r, tokens in enumerate(tokenized_text):
        feature = {}
        for token in tokens:
            word_index = vocabulary_.get(token)
            if word_index is not None:
                if word_index not in feature.keys():
                    feature[word_index] = 1
                else:
                    feature[word_index] += 1
        for c, v in feature.items():
            values.append(v)
            row_indices.append(r)
            col_indices.append(c)
        #print(feature)

    # document-term matrix in sparse CSR format
    X = sp.csr_matrix((values, (row_indices, col_indices)),
                      shape=(n_doc, len(vocabulary_)))
    return X

vocabulary_ = {v: k for k, v in enumerate(set(chain.from_iterable(tokenized_texts)))}
X = text_to_bow(tokenized_texts, vocabulary_)

In [25]:
transformer = TfidfTransformer()
svd_model = TruncatedSVD(n_components=100,
                         algorithm='arpack', n_iter=100)
X_tfidf = transformer.fit_transform(X)
X_svd = svd_model.fit_transform(X_tfidf)

In [26]:
tag = pd.get_dummies(data.Category).columns

In [27]:
import joblib
#Load Model
logist_models = joblib.load("category_model.pkl")

/home/jupyter-e22vvb/.local/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [28]:
y_pred = np.argmax(np.vstack([model.predict_proba(X_svd)[:, 1] for model in logist_models]).T, axis=1)
y_pred = np.array([tag[yi] for yi in y_pred])
y_true = data.Category.values
print(tag[0:4])

Index(['การรับเข้านักศึกษา', 'คำถามทั่วไป', 'ลงทะเบียนเรียน', 'หลักสูตร'], dtype='object')


In [29]:
#Clean Text
def remove_repettition(text):
    token_list = list(text)
    if len(token_list) > 2:
        filter_list = [True, True]
        n = len(token_list)
        for i in range(2, n):
            if (token_list[i] == token_list[i-1]) and (token_list[i] == token_list[i-2]):
                filter_list.append(False)
            else:
                filter_list.append(True)

        output = ''.join(np.array(token_list)[filter_list])
    else:
        output = text
    return output

def cleansing(text):
    # \t, \n, \xa0 and other special characters. Replace by blank string
    text = re.sub('[\t\n\xa0\"\'!?\/\(\)%\:\=\-\+\*\_ๆ]', '', text)
    
    # Numbers. Replace by space
    text = re.sub('[0-9]', ' ', text)
    
    # Dot. Replace by space
    text = re.sub('[\.]', ' ', text)
    
    # One or more consecutive space. Replace by single space
    text = re.sub('\s+',' ',text)
    
    # Remove 2 or more repettition
    text = remove_repettition(text)
    
    return text

In [17]:
textInput = 'วิศวคอมพิวเตอร์มีหลักสูตรอะไรบ้าง'
textInput= cleansing(textInput)
tokenized_text = deepcut.tokenize(textInput)
x = text_to_bow([tokenized_text], vocabulary_)
x_tfidf = transformer.transform(x)
x_svd = svd_model.transform(x_tfidf)
pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
print(list(zip(tag, pred)))
predict_category = max(list(zip(tag, pred)))

[('การรับเข้านักศึกษา', 0.15501360454034838), ('คำถามทั่วไป', 0.23211256402801864), ('ลงทะเบียนเรียน', 0.08012019772465477), ('หลักสูตร', 0.4839496158840357)]


In [18]:
max_value = 0
max_category = ''
pred_results = list(zip(tag, pred))
for pred_result in pred_results:
  # print(pred_result)
  if pred_result[1] > max_value:
    max_value = pred_result[1]
    max_category = pred_result[0]
print(max_category, max_value)

หลักสูตร 0.4839496158840357


## prediction

In [19]:
#cosine
from pythainlp import word_tokenize 
from pythainlp.word_vector import * 
from sklearn.metrics.pairwise import cosine_similarity  
import numpy as np
model=get_model()
def sentence_vectorizer(ss,dim=300,use_mean=True): 
    s = word_tokenize(ss)
    vec = np.zeros((1,dim))
    for word in s:
        if word in model.wv.index2word:
            vec+= model.wv.word_vec(word)
        else: pass
    if use_mean: vec /= len(s)
    return vec
def sentence_similarity(s1,s2):
    return cosine_similarity(sentence_vectorizer(str(s1)),sentence_vectorizer(str(s2)))

In [20]:
len(beforeTok[max_category])

82

In [21]:
word = beforeTok[max_category].iloc[0]

In [22]:
word[0]

'วิชาภาคบังคับที่มีวิชาต่อเนื่อง มีอะไรบ้าง หลักสูตรปกติ?'

In [23]:
word

curriculum    วิชาภาคบังคับที่มีวิชาต่อเนื่อง มีอะไรบ้าง หลั...
Name: 0, dtype: object

In [24]:
import json

In [25]:
result = []
for index in range(len(beforeTok[max_category])):
#     print(beforeTok[max_category].iloc[index])
    word = beforeTok[max_category].iloc[index]
#     print(word[0])
#     word = beforeTok[max_category].iloc[index]
    subResult = sentence_similarity(textInput,word[0])
    subResult = subResult[0][0]
    result.append(subResult)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self i

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self i

In [26]:
max_question_percentage = max(result)
print(max_question_percentage)
question_index = np.where(result == max_question_percentage)
print(question_index)
predictedQuestion = beforeTok[max_category].loc[question_index[0][0]]
predictedQuestion = predictedQuestion[0]

0.7615871650925202
(array([74]),)


In [27]:
predictedQuestion

'วิชาภาคบังคับที่มีตัวต่อเนื่องมีอะไรบ้าง หลักสูตรนานาชาติ?'

## run api

In [ ]:
app = Flask(__name__)
@app.route('/prediction', methods=['POST', 'GET'])
def home():
    if request.method == 'POST':
        data = request.get_json()
        textInput = data["inputQuestion"]
        textInput= cleansing(textInput)
        tokenized_text = deepcut.tokenize(textInput)
        x = text_to_bow([tokenized_text], vocabulary_)
        x_tfidf = transformer.transform(x)
        x_svd = svd_model.transform(x_tfidf)
        pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
        print(list(zip(tag, pred)))
        predict_category = max(list(zip(tag, pred)))

        max_value = 0
        max_category = ''
        pred_results = list(zip(tag, pred))
        for pred_result in pred_results:
          # print(pred_result)
            if pred_result[1] > max_value:
                max_value = pred_result[1]
                max_category = pred_result[0]
        print(max_category, max_value)

        result = []
        for index in range(len(beforeTok[max_category])):
            word = beforeTok[max_category].iloc[index]
            subResult = sentence_similarity(textInput,word[0])
            subResult = subResult[0][0]
            result.append(subResult)
        max_question_percentage = max(result)
        print(max_question_percentage)
        question_index = np.where(result == max_question_percentage)
        print(question_index)
        #         print(beforeTok[max_category].loc[question_index[0][0]])
        predictedQuestion = beforeTok[max_category].loc[question_index[0][0]]
        predictedQuestion = predictedQuestion[0]
        print(predictedQuestion)
        value = {
            "category": max_category,
            "accuracy": "%lf" % max_value,
            "predictedQuestion": str(predictedQuestion),
            "similarity": "%lf" % max_question_percentage
        }
        return json.dumps(value, ensure_ascii=False).encode('utf8')
    else:
        return "Hello. I am alive!"

app.run(port=5000,debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


# example api

In [ ]:
{
"inputQuestion":"วิศวคอมมีหลักสูตรอะไรบ้าง"
}